# About: ロギングドライバを変更する
---

`docker-compose.yml`を編集してMoodleコンテナのロギングドライバを変更します。

## 概要

Dockerコンテナのロギングドライバをデフォルトの `json-file` から `syslog` に変更します。

![処理の流れ](images/moodle-041-01.png)

ロギングドライバの変更は「040-設定ファイルの変更.ipynb」と同様の手順で`docker-compose.yml`を編集することで行います。主な変更手順を以下に示します。

1. 構築環境に配置されている`docker-compose.yml`をローカルのJupyter Notebook環境に取得する
2. 取得したファイルのバックアップを作成する
3. Notebookの編集機能を利用して`docker-compose.yml`の変更をおこなう
4. 変更した`docker-compose.yml`を構築環境に配置する
5. `docker-compose.yml`の変更を反映させるためにコンテナを再起動する

### 前提条件

* Moodle実行環境とは別のノードに syslogサーバが構築済であること
* syslogサーバはMoodle環境からのログを受け取れるように設定されていること

## パラメータの指定

### グループ名の指定

このNotebookの操作対象となるAnsibleのグループ名を設定します。

In [ ]:
# (例)
# target_group = 'Moodle'

target_group =

#### チェック

指定された `target_group` の値が適切なものかチェックします。

`target_group` に対応する `group_vars` ファイルが存在していることを確認します。

In [ ]:
from pathlib import Path
if not (Path('group_vars') / (target_group + '.yml')).exists():
    raise RuntimeError(f"ERROR: not exists {target_group + '.yml'}")

`target_group`で指定したホストにAnsibleで到達可能であることを確認します。

In [ ]:
!ansible {target_group} -m ping

### syslogサーバの確認

syslogサーバをチェックするために Moodle構築環境からテスト用のログを送信してみます。

`logger` コマンドを用いてテスト用のログを送信してみます。

syslogサーバのアドレスを指定してください。

In [ ]:
# (例)
# syslog_server = 'syslog.example.org'
# syslog_server = '192.168.200.200'

syslog_server =

syslogのポート番号を指定してください。

In [ ]:
# (例)
# syslog_port = 514

syslog_port = 514

`logger` コマンドのオプションを指定してください。指定可能なオプションを以下に示します。

```
 -T, --tcp             use TCP only
 -d, --udp             use UDP only
 -i, --id              log the process ID too
 -f, --file <file>     log the contents of this file
 -h, --help            display this help text and exit
 -S, --size <num>      maximum size for a single message (default 1024)
 -n, --server <name>   write to this remote syslog server
 -P, --port <port>     use this port for UDP or TCP connection
 -p, --priority <prio> mark given message with this priority
 -s, --stderr          output message to standard error as well
 -t, --tag <tag>       mark every line with this tag
 -u, --socket <socket> write to this Unix socket
 -V, --version         output version information and exit
```

In [ ]:
# (例)
# logger_opts = '-T'  # TCPの場合
# logger_opts = '-d'  # UDPの場合

logger_opts =

テスト用のメッセージを指定してください。

In [ ]:
# (例)
# logger_message = 'test message 001'

logger_message = 'test message 001'

Moodle構築環境からテスト用のログを送信してみます。

In [ ]:
!ansible {target_group} -a \
    'logger -n {syslog_server} -P {syslog_port} {logger_opts} "{logger_message}"'

syslogサーバでログを受け取ることができているかを確認してください。

## docker-compose.yml の編集

`docker-compose.yml`をローカル環境に取得してMoodleコンテナのロギングドライバを変更します。

![設定ファイルの取得、編集](images/moodle-040-03.png)

次のセルを実行すると、以下の手順が実行されます。

1. 構築環境に配置されている`docker-compose.yml`をローカル環境に取得する
2. 取得した`docker-compose.yml`のバックアップを作成する
3. Jupyter Notebookの編集機能を利用して設定ファイルを編集するためのリンクを表示する

In [ ]:
%run scripts/edit_conf.py
fetch_docker_compose(target_group)

上のセルの出力に表示されているリンクをクリックして設定ファイルの編集を行います。

コンテナのログをsyslogに転送するために `docker-compose.yml` に対して以下の変更を行います。

1. コンテナのロギングドライバを syslog に変更する
1. Moodleコンテナのログファイル出力ディレクトリへのbind mountを無効にする
1. MariaDBのログ出力先をファイルからコンソールに変更するようにコマンドラインオプションの指定を行う

Moodleコンテナのロギングドライバを syslog に変更するには`docker-compose.yml` の `moodle`サービスに以下のような設定を追加します。


```
    logging:
      driver: syslog
      options:
        syslog-address: "{tcp|udp}://{syslogサーバのアドレス}:{syslogサーバのポート番号}"
        tag: moodle
```

次のセルを実行すると前章で設定したパラメータにあわせた設定内容を表示します。上のセルのリンクから開いた `docker.compose.yml` の編集画面に追記して保存してください。YAMLではインデントに意味をもたせているのでスペース文字数に注意して編集してください。

In [ ]:
syslog_proto = 'tcp' if logger_opts.find('-T') >= 0 else 'udp'
print(f'''
    logging:
      driver: syslog
      options:
        syslog-address: "{syslog_proto}://{syslog_server}:{syslog_port}"
        tag: moodle
''')

同様の変更をdbコンテナに対しても行ってください。ただしログでコンテナ種別を判別するために `tag` の指定にMoodleコンテナとは別の値を指定してください。

`docker-compose.yml` のロギング設定の記述方法の詳細については以下の記述などを参照してください。

* [Compose file version 3 reference](https://docs.docker.com/compose/compose-file/#logging)
* [Syslog logging driver](https://docs.docker.com/config/containers/logging/syslog/)

`docker-compose.yml` を編集した後に、次のセルを実行すると編集の前後での差分を確認することができます。

In [ ]:
show_local_docker_compose_diff(target_group)

ここまでの変更に関する差分の表示例を以下に示します。

```
--- docker-compose.yml.orig
+++ docker-compose.yml
@@ -19,6 +19,11 @@
       - --collation-server=utf8mb4_unicode_ci
       - --explicit_defaults_for_timestamp=true
       - --log-error=/var/log/mysql/error.log
+    logging:
+      driver: syslog
+      options:
+        syslog-address: "tcp://syslog.example.org:514"
+        tag: mariadb
   moodle:
     image: harbor.vcloud.nii.ac.jp/vcp/moodle-simple:3.9
     container_name: moodle
@@ -48,3 +53,8 @@
       DB_PASSWORD: "moodle"
       MOODLE_REVERSEPROXY: "true"
       WAIT_FOR: db:3306
+    logging:
+      driver: syslog
+      options:
+        syslog-address: "tcp://syslog.example.org:514"
+        tag: moodle
```

次にMoodleコンテナのログファイル出力ディレクトリへのbind mountを無効にします。Dockerのロギングドライバでsyslogへの転送を行うには、MoodleコンテナのApache HTTP Serverのアクセスログなどをファイルとして出力するのではなく、コンテナのログとして扱えるようにする必要があります。Moodleコンテナでは `/var/log/httpd/` への bind mount を無効にすることによってファイル出力ではなくコンテナのログとして処理されます。これによりロギングドライバによる syslog への転送が可能となります。

`docker-compose.yml` における `moodle` コンテナの `volumes` 設定から `/var/log/httpd` のエントリを削除してください。編集後に次のセルを実行すると編集の前後での差分を確認することができます。

In [ ]:
show_local_docker_compose_diff(target_group)

ここまでの変更に関する差分の表示例を以下に示します。

```
--- docker-compose.yml.orig
+++ docker-compose.yml
@@ -19,6 +19,11 @@
       - --collation-server=utf8mb4_unicode_ci
       - --explicit_defaults_for_timestamp=true
       - --log-error=/var/log/mysql/error.log
+    logging:
+      driver: syslog
+      options:
+        syslog-address: "tcp://syslog.example.org:514"
+        tag: mariadb
   moodle:
     image: harbor.vcloud.nii.ac.jp/vcp/moodle-simple:3.9
     container_name: moodle
@@ -32,7 +37,6 @@
       - /srv/moodle/moodle/conf/httpd/conf.d:/etc/httpd/conf.d
       - /srv/moodle/moodle/conf/httpd/conf.modules.d:/etc/httpd/conf.modules.d
       - /srv/moodle/moodle/conf/ca-trust:/etc/pki/ca-trust/source/anchors
-      - /var/log/httpd:/var/log/httpd
     ports:
       - 80:80
     environment:
@@ -48,3 +52,8 @@
       DB_PASSWORD: "moodle"
       MOODLE_REVERSEPROXY: "true"
       WAIT_FOR: db:3306
+    logging:
+      driver: syslog
+      options:
+        syslog-address: "tcp://syslog.example.org:514"
+        tag: moodle
```

最後にMariaDBのログ出力先をファイルからコンソールに変更するようにコマンドラインオプションの指定を行います。ログの出力先をファイルからコンソールに変更しコンテナのログとして扱えるようにします。

`docker-compose.yml` における `db` コンテナの `command` 設定を変更します。`--log-error=` の行を削除し、替わりに `--console` の指定を追加してください。

編集後に次のセルを実行すると編集の前後での差分を確認することができます。

In [ ]:
show_local_docker_compose_diff(target_group)

ここまでの変更に関する差分の表示例を以下に示します。

```
--- docker-compose.yml.orig
+++ docker-compose.yml
@@ -18,7 +18,12 @@
       - --character-set-server=utf8mb4
       - --collation-server=utf8mb4_unicode_ci
       - --explicit_defaults_for_timestamp=true
-      - --log-error=/var/log/mysql/error.log
+      - --console
+    logging:
+      driver: syslog
+      options:
+        syslog-address: "tcp://syslog.example.org:514"
+        tag: mariadb
@@ -32,7 +37,6 @@
       - /srv/moodle/moodle/conf/httpd/conf.d:/etc/httpd/conf.d
       - /srv/moodle/moodle/conf/httpd/conf.modules.d:/etc/httpd/conf.modules.d
       - /srv/moodle/moodle/conf/ca-trust:/etc/pki/ca-trust/source/anchors
-      - /var/log/httpd:/var/log/httpd
     ports:
       - 80:80
     environment:
@@ -48,3 +52,8 @@
       DB_PASSWORD: "moodle"
       MOODLE_REVERSEPROXY: "true"
       WAIT_FOR: db:3306
+    logging:
+      driver: syslog
+      options:
+        syslog-address: "tcp://syslog.example.org:514"
+        tag: moodle
```

## 編集した設定ファイルの反映

設定ファイルの変更内容をコンテナに反映させ、実際にMoodleコンテナのロギングドライバをsyslogに変更します。

![設定ファイルの反映](images/moodle-031-04.png)

### 編集内容の反映

前章で編集した`docker-compose.yml`を構築環境に配置して設定を反映させます。

次のセルを実行すると、以下の手順が実行されます。

1. 編集前と編集後の`docker-compose.yml`の差分を表示する
2. 編集した`docker-compose.yml`を構築環境に配置する
3. コンテナを再起動して変更した`docker-compose.yml`の反映を行う

In [ ]:
apply_docker_compose(target_group)

syslogサーバにコンテナのログが送信されるようになったことを確認してください。